In [1]:
#https://population.un.org/wpp/Download/Standard/CSV/
#https://github.com/novelcovid/api

import pandas as pd
import json
import requests

## World - Countries aggretation
### Fetching countries geo-data from corona.lmao.ninja's API

In [2]:
url = "https://corona.lmao.ninja/countries?sort=country"
req = requests.get(url)
j = json.loads(req.text)

# getting countries names for reference
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Parsing countries data do dict, then pandas dataframe
dic = []
i = 0
for country in countries:
    row = {'country': j[i]['country'], '_id': j[i]['countryInfo']['_id'], 'iso2': j[i]['countryInfo']['iso2'], 'iso3': j[i]['countryInfo']['iso3'],  
       'lat':j[i]['countryInfo']['lat'], 'long': j[i]['countryInfo']['long'], 'flag':j[i]['countryInfo']['flag']}
                 
    dic.append(row)
    i += 1 
df = pd.DataFrame.from_dict(dic)
df.tail()

,country,_id,iso2,iso3,lat,long,flag
207,Angola,24.0,AO,AGO,-12.5,18.5,https://raw.githubusercontent.com/NovelCOVID/A...
208,Andorra,20.0,AD,AND,42.5,1.6,https://raw.githubusercontent.com/NovelCOVID/A...
209,Algeria,12.0,DZ,DZA,28.0,3.0,https://raw.githubusercontent.com/NovelCOVID/A...
210,Albania,8.0,AL,ALB,41.0,20.0,https://raw.githubusercontent.com/NovelCOVID/A...
211,Afghanistan,4.0,AF,AFG,33.0,65.0,https://raw.githubusercontent.com/NovelCOVID/A...


### Fecthing countries data from UN's API
#### 2019 data

In [3]:
df_un= pd.read_csv('../data/WPP2019_TotalPopulationBySex.csv')
df_un = df_un[df_un['Time']==2019]
df_un.tail()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
277314,900,World,2,Medium,2019,2019.5,3889034.611,3824433.594,7713468.205,59.291
278198,1801,World Bank Regional Groups (developing only),2,Medium,2019,2019.5,3261387.881,3191129.174,6452517.055,70.258
278349,887,Yemen,2,Medium,2019,2019.5,14692.284,14469.638,29161.922,55.234
279233,894,Zambia,2,Medium,2019,2019.5,8843.214,9017.820,17861.034,24.026
280117,716,Zimbabwe,2,Medium,2019,2019.5,6983.353,7662.120,14645.473,37.858


In [10]:
# normalizing unmatchables country names
df_un.loc[df_un['Location']=='United States of America (and dependencies)', 'Location'] = 'US'
df_un.loc[df_un['Location']=='Bolivia (Plurinational State of)', 'Location'] = 'Bolivia'
df.loc[df['country']=='UK', 'country'] = 'United Kingdom'
df.loc[df['country']=='USA', 'country'] = 'US'

df_un.loc[df_un['Location']=='France']

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
93672,250,France,2,Medium,2019,2019.5,31524.494,33605.237,65129.731,118.946


#### Merging both dataframes and saving to csv

In [5]:
df_countries = pd.merge(df, df_un, how='inner', on=None, left_on='country', 
                 right_on='Location', left_index=False, right_index=False, sort=False)   

df_countries.drop(['_id', 'Location', 'MidPeriod'], axis=1, inplace=True)
df_countries.tail()

,country,iso2,iso3,lat,long,flag,LocID,VarID,Variant,Time,PopMale,PopFemale,PopTotal,PopDensity
181,Angola,AO,AGO,-12.5,18.5,https://raw.githubusercontent.com/NovelCOVID/A...,24,2,Medium,2019,15744.779,16080.520,31825.299,25.528
182,Andorra,AD,AND,42.5,1.6,https://raw.githubusercontent.com/NovelCOVID/A...,20,2,Medium,2019,NaN,NaN,77.146,164.140
183,Algeria,DZ,DZA,28.0,3.0,https://raw.githubusercontent.com/NovelCOVID/A...,12,2,Medium,2019,21749.666,21303.388,43053.054,18.076
184,Albania,AL,ALB,41.0,20.0,https://raw.githubusercontent.com/NovelCOVID/A...,8,2,Medium,2019,1466.785,1414.128,2880.913,105.143
185,Afghanistan,AF,AFG,33.0,65.0,https://raw.githubusercontent.com/NovelCOVID/A...,4,2,Medium,2019,19529.727,18512.030,38041.757,58.269


In [6]:
df_countries.to_csv('../data/world_countries_2019.csv',  index = False)

In [7]:
df_un['Location'].unique()

array(['Afghanistan', 'Africa', 'African Group', 'African Union',
       'African Union: Central Africa', 'African Union: Eastern Africa',
       'African Union: Northern Africa', 'African Union: Southern Africa',
       'African Union: Western Africa',
       'African, Caribbean and Pacific (ACP) Group of States', 'Albania',
       'Algeria', 'American Samoa', 'Andean Community', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Asia',
       'Asia-Pacific Economic Cooperation (APEC)', 'Asia-Pacific Group',
       'Association of Southeast Asian Nations (ASEAN)', 'Australia',
       'Australia/New Zealand', 'Austria', 'Azerbaijan', 'BRIC', 'BRICS',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Belt-Road Initiative (BRI)',
       'Belt-Road Initiative: Africa', 'Belt-Road Initiative: Asia',
       'Belt-Road Initiative: Europe',
       'Belt-Road Initiative: Latin America and the Cari

## Brazil - States aggretation
### Fetching states geo-data from IBGE's API

In [8]:
#https://servicodados.ibge.gov.br/api/docs/localidades
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}
estado_jsons = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/?formato=application/vnd.geo+json&resolucao=2",
                           headers=headers)
estado_json = estado_jsons.json()

f = open('../data/brasil-estados.json', 'w')
json.dump(estado_json, f)
f.close()

estados = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/",
                       headers=headers).json()
dic = []
for estado in estados:
    linha = {'id_estado':estado['id'], 'nome_estado': estado['nome'], 'sigla_estado': estado['sigla'],
              'id_regiao':estado['regiao']['id'],  'nome_regiao':estado['regiao']['nome'], 'sigla_regiao':estado['regiao']['sigla']}
    dic.append(linha)
        
df_estados = pd.DataFrame.from_dict(dic)
df_estados.head()
df_estados.to_csv('../data/brazilian_states.csv',  index = False)

In [9]:
print('Countries aggregation done!')

Countries aggregation done!
